In [1]:
# Train the ML model

from models import extendedCED, mse_loss_encoder, mse_loss_decoder
from utils import load_model_data_new, unnormalize_params, assess_decoder
from utils import plot_loss, normalize_params, load_encoder_data
import time
import glob
from telnetlib import EC
import tensorflow as tf
import yaml
import os
import numpy as np
from datetime import datetime
import argparse
import matplotlib.pyplot as plt
import matplotlib as mpl


2022-09-07 13:57:51.666684: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-07 13:57:51.669970: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kiliakis/install/lib:/usr/lib/x86_64-linux-gnu
2022-09-07 13:57:51.669980: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Initialize parameters
data_dir = '/eos/kiliakis/tomo_data/datasets'
timestamp = datetime.now().strftime("%Y_%m_%d_%H-%M-%S")

# Data specific
IMG_OUTPUT_SIZE = 128
BUFFER_SIZE = 1000
BATCH_SIZE = 32  # 8
latent_dim = 7  # 6 + the new VrfSPS
additional_latent_dim = 1

# Train specific
models_to_train = ['encoder']
train_cfg = {
    'encoder': {
        'epochs': 2,
        'lr': 2e-4,
    },
}

# Keep only a small percentage of the entire dataset
# for faster testing.
dataset_keep_percent = 0.1
# cnn_filters = [32, 64, 128, 256, 512, 1024]
cnn_filters = [32]


In [3]:
# Initialize directories
trial_dir = os.path.join('./trials/', timestamp)
weights_dir = os.path.join(trial_dir, 'weights')
plots_dir = os.path.join(trial_dir, 'plots')

# Initialize train/ test / validation paths
ML_dir = os.path.join(data_dir, 'ML_data')
TRAINING_PATH = os.path.join(ML_dir, 'TRAINING')
VALIDATION_PATH = os.path.join(ML_dir, 'VALIDATION')
# TESTING_PATH = os.path.join(ML_dir, 'TESTING')
assert os.path.exists(TRAINING_PATH)
assert os.path.exists(VALIDATION_PATH)
# assert os.path.exists(TESTING_PATH)

# create the directory to store the results
os.makedirs(trial_dir, exist_ok=True)
os.makedirs(weights_dir, exist_ok=False)
os.makedirs(plots_dir, exist_ok=False)


In [6]:
# Create the datasets
files = glob.glob(TRAINING_PATH + '/*.pk')
files = files[:int(len(files) * dataset_keep_percent)]
# train_dataset = tf.data.Dataset.list_files(TRAINING_PATH + '/*.pk')
train_dataset = tf.data.Dataset.from_tensor_slices(files)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.map(lambda x: tf.py_function(load_encoder_data, [x], (tf.float32,
                                                                                    [tf.float32, tf.float32, tf.float32, tf.float32,
                                                                                     tf.float32, tf.float32, tf.float32]))

# train_dataset = train_dataset.map(
#     lambda x: tf.py_function(load_model_data_new,
                            #  [x],
                            #  [tf.float32, tf.float32, tf.float32, tf.string,
                            #   tf.float32, tf.float32, tf.float32, tf.float32,
                            #   tf.float32, tf.float32, tf.float32, tf.float32,
                            #   tf.float32]))
# train_dataset = train_dataset.map(lambda *x: x[1])
train_dataset = train_dataset.map(
    lambda *x: (x[1], tf.convert_to_tensor(x[4:11])))
# y_train = train_dataset.map(lambda *x: normalize_params(*x[4:11]))

files = glob.glob(VALIDATION_PATH + '/*.pk')
files = files[:int(len(files) * dataset_keep_percent)]
# valid_dataset = tf.data.Dataset.list_files(VALIDATION_PATH + '/*.pk')
valid_dataset = tf.data.Dataset.from_tensor_slices(files)
valid_dataset = valid_dataset.shuffle(BUFFER_SIZE)
# valid_dataset = valid_dataset.batch(BATCH_SIZE)
# valid_dataset = valid_dataset.map(lambda x: tf.py_function(load_model_data_new, [x],
valid_dataset = valid_dataset.map(
    lambda x: tf.py_function(load_encoder_data, [x], (tf.float32,
    [tf.float32, tf.float32, tf.float32, tf.float32,
     tf.float32, tf.float32, tf.float32]))

# valid_dataset=valid_dataset.map(lambda x: tf.py_function(load_encoder_data, [x],
#                                                            [tf.float32, tf.float32, tf.float32, tf.string,
#                                                             tf.float32, tf.float32, tf.float32, tf.float32,
#                                                             tf.float32, tf.float32, tf.float32, tf.float32,
#                                                             tf.float32]))
# valid_dataset = valid_dataset.map(lambda *x: (x[1], tf.convert_to_tensor(x[4:11])))
# y_valid = valid_dataset.map(lambda *x: normalize_params(*x[4:11]))


SyntaxError: invalid syntax (2920153100.py, line 20)

In [11]:
for features, targets in valid_dataset:
    print(targets)
    break

tf.Tensor([ 6.00e+00 -1.40e+01  1.36e-09  5.70e+10  7.90e+00  4.10e+00  8.30e+00], shape=(7,), dtype=float32)


In [13]:
# Model instantiation
input_shape = (IMG_OUTPUT_SIZE, IMG_OUTPUT_SIZE, 1)

eCED = extendedCED(latent_dim, additional_latent_dim, input_shape,
                   filters=cnn_filters)

print(eCED.encoder.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 63, 63, 32)        320       
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 7)                 889063    
                                                                 
Total params: 889,383
Trainable params: 889,383
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# Train the encoder

optimizer = tf.keras.optimizers.Adam(train_cfg['encoder']['lr'])

eCED.encoder.compile(optimizer=optimizer, loss='mse')
history = eCED.encoder.fit(
    train_dataset, epochs=train_cfg['encoder']['epochs'],
    validation_data=valid_dataset)


Epoch 1/2


: 

: 